In [19]:
import torch
from transformers import AutoModel, AutoTokenizer,AutoModelForMaskedLM, AutoModelForCausalLM,AutoModelForSeq2SeqLM,GraphormerForGraphClassification
import pubchempy as pcp
from scipy.io import loadmat
import pandas as pd
import numpy as np 
from rdkit import Chem
import os
from __future__ import annotations
import os
from typing import Optional, Dict, Tuple, List

import numpy as np
import pandas as pd
import torch
from __future__ import annotations
import os, torch
import pandas as pd
from typing import Optional
import os
import torch
import pandas as pd
import numpy as np
import os
import pandas as pd
import numpy as np

from rdkit import Chem, DataStructs
from rdkit.Chem import AllChem, RDKFingerprint, MACCSkeys
import os
import pandas as pd
import numpy as np

from rdkit import Chem
from rdkit.Chem import Descriptors
from rdkit.ML.Descriptors import MoleculeDescriptors

In [16]:
base_dir = '../../../../KINGSTON/DATASETS'
base_dir = '../data'

In [3]:
! ls '../../../../KINGSTON/DATASETS'

attention  datasets   embeddings folds


In [ ]:
# set_seeds(seed=2024)

NameError: name 'set_seeds' is not defined

In [17]:
import os
import json
import torch
import pandas as pd

def extract_tokens(
    tokenizer,
    model,              # kept for API symmetry, but not actually used here
    model_name: str,   
    ds: str,
    input_type: str = "smiles",   # "smiles" or "selfies"
    save_path: str | None = None,
) -> pd.DataFrame:
    """
    Extract *token IDs* from UNIQUE CIDs using either SMILES or SELFIES.

    Output columns:
        ['cid', 'isomeric_text', 'canonical_text',
         'input_type', 'model', 'iso_token_ids', 'can_token_ids']

    - iso_token_ids, can_token_ids are JSON-encoded lists of ints
      corresponding to tokenizer(...)[\"input_ids\"].
    """

    assert input_type.lower() in {"smiles", "selfies"}, "input_type must be 'smiles' or 'selfies'"
    print(f"Extracting {model_name} TOKENS for {ds} using {input_type}...")

    # ---- Load & pick columns ----
    df = pd.read_csv(f"{base_dir}/datasets/{ds}/{ds}_data.csv")
    if "cid" not in df.columns:
        raise ValueError("Dataset must contain a 'cid' column.")

    if input_type.lower() == "smiles":
        iso_col = "isomericsmiles"
        can_col = "canonicalsmiles"
    else:  # selfies
        iso_col = "isomericselfies"
        can_col = "canonicalselfies"

    if iso_col is None and can_col is None:
        raise ValueError(f"No {input_type} columns found.")

    use_cols = ["cid"]
    if iso_col: use_cols.append(iso_col)
    if can_col: use_cols.append(can_col)

    work = df[use_cols].copy()
    work = (
        work
        .drop_duplicates(subset=["cid"], keep="first")
        .sort_values("cid")
        .reset_index(drop=True)
    )

    rows = []

    for idx, row_in in work.iterrows():
        cid = row_in["cid"]
        iso_txt = row_in.get(iso_col, "")
        can_txt = row_in.get(can_col, "")

        print(f"Processing CID {cid}...")

        # --- normalize texts ---
        iso_txt = iso_txt if isinstance(iso_txt, str) and iso_txt.strip() else ""
        can_txt = can_txt if isinstance(can_txt, str) and can_txt.strip() else ""
        if not iso_txt and not can_txt:
            # nothing to encode for this cid
            continue

        iso_ids = None
        can_ids = None

        # --- tokenize isomeric (if present) ---
        if iso_txt:
            iso_inputs = tokenizer(
                [iso_txt],
                padding=True,
                truncation=True,
                return_tensors="pt"
            )
            # (1, T) -> list of ints
            iso_ids = iso_inputs["input_ids"][0].tolist()

        # --- tokenize canonical (if present) ---
        if can_txt:
            can_inputs = tokenizer(
                [can_txt],
                padding=True,
                truncation=True,
                return_tensors="pt"
            )
            can_ids = can_inputs["input_ids"][0].tolist()

        out_row = {
            "cid": cid,
            "isomeric_text": iso_txt,
            "canonical_text": can_txt,
            "input_type": input_type.lower(),   # "smiles" or "selfies"
            "model": model_name,
            # store lists as JSON strings so lengths can vary per molecule
            "iso_token_ids": json.dumps(iso_ids) if iso_ids is not None else "",
            "can_token_ids": json.dumps(can_ids) if can_ids is not None else "",
        }

        rows.append(out_row)

    out_df = pd.DataFrame(rows)

    if save_path is not None:
        os.makedirs(save_path, exist_ok=True)
        out_file = f"{save_path}/{ds}_{model_name.split('/')[1]}_tokens.csv"
        print(f"Saving to {out_file}")
        out_df.to_csv(out_file, index=False)

    return out_df


In [1]:
def extract_representations(
    tokenizer,
    model,
    model_name: str,   
    ds: str,
    input_type: str = "smiles",   # "smiles" or "selfies"
    token: int = 0,               # which token to read (e.g., CLS/first)
    save_path: str | None = None,
    mean : bool = False
) -> pd.DataFrame:
    """
    Create embeddings from UNIQUE CIDs using either SMILES or SELFIES.
    The output DF has:
        ['cid', 'isomeric_text', 'canonical_text', 'input_type', 'model', 'layer', 'e0', ..., 'e{d-1}'].

    Tokenization uses the chosen input_type; for each row it prefers the isomeric text if present,
    otherwise falls back to the canonical text. Both text variants are kept as metadata columns.
    """
    assert input_type.lower() in {"smiles", "selfies"}, "input_type must be 'smiles' or 'selfies'"
    model.eval()
    print(f"Extracting {model_name} embeddings for {ds} using {input_type}...")

    # ---- Load & pick columns ----
    df = pd.read_csv(f"{base_dir}/datasets/{ds}/{ds}_data.csv")
    if "cid" not in df.columns:
        raise ValueError("Dataset must contain a 'cid' column.")

    if input_type.lower() == "smiles":
        iso_col = "isomericsmiles"
        can_col = "canonicalsmiles"
    else:  # selfies
        iso_col = "isomericselfies"
        can_col = "canonicalselfies"

    if iso_col is None and can_col is None:
        raise ValueError(f"No {input_type} columns found.")

    use_cols = ["cid"]
    if iso_col: use_cols.append(iso_col)
    if can_col: use_cols.append(can_col)

    work = df[use_cols].copy()
    work = work.drop_duplicates(subset=["cid"], keep="first").sort_values("cid").reset_index(drop=True)

   
    # ---- Device ----
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    rows = []
    emb_dim = None

    for cid, iso_txt, can_txt in zip(work["cid"], work[iso_col], work[can_col]):
        print(f"Processing CID {cid}...")
        # --- normalize texts ---
        iso_txt = iso_txt if isinstance(iso_txt, str) and iso_txt.strip() else ""
        can_txt = can_txt if isinstance(can_txt, str) and can_txt.strip() else ""
        if not iso_txt and not can_txt:
            continue  # nothing to encode for this cid

        # --- run model for isomeric (if present) ---
        iso_hiddens = None
        if iso_txt:
            iso_inputs = tokenizer([iso_txt], padding=True, truncation=True, return_tensors="pt")
            iso_inputs = {k: v.to(device) for k, v in iso_inputs.items()}
            with torch.no_grad():
                iso_out = model(**iso_inputs, output_hidden_states=True)
            iso_hiddens = iso_out.hidden_states  # tuple of [B,T,D] tensors

        # --- run model for canonical (if present) ---
        can_hiddens = None
        if can_txt:
            can_inputs = tokenizer([can_txt], padding=True, truncation=True, return_tensors="pt")
            can_inputs = {k: v.to(device) for k, v in can_inputs.items()}
            with torch.no_grad():
                can_out = model(**can_inputs, output_hidden_states=True)
            can_hiddens = can_out.hidden_states

        # number of layers to emit = max of the two (they should match)
        n_layers = max(
            len(iso_hiddens) if iso_hiddens is not None else 0,
            len(can_hiddens) if can_hiddens is not None else 0,
        )

        for layer_idx in range(n_layers):
            # get vectors (or None) for this layer
            # print(f"Processing CID {cid}, layer {layer_idx+1}/{n_layers}")
           
            if mean:
                iso_vec = iso_hiddens[layer_idx][0, :].mean(dim=0).detach().cpu().numpy()
                can_vec = can_hiddens[layer_idx][0, :].mean(dim=0).detach().cpu().numpy()
            else:
                iso_vec = iso_hiddens[layer_idx][0, token, :].detach().cpu().numpy()
                can_vec = can_hiddens[layer_idx][0, token, :].detach().cpu().numpy()    

            # set / check embedding dim
            if emb_dim is None:
                
                emb_dim = iso_vec.shape[0]
                emb_dim = can_vec.shape[0]
            # sanity: if both exist, ensure same D
            if (iso_vec is not None) and (can_vec is not None):
                assert iso_vec.shape[0] == can_vec.shape[0], "Iso/Can dims differ!"

            row = {
                "cid": cid,
                "isomeric_text": iso_txt,
                "canonical_text": can_txt,
                "input_type": input_type.lower(),   # "smiles" or "selfies"
                "model": model_name,
                "layer": layer_idx,
            }

            # add iso_* columns (fill with NaN if missing)
            if emb_dim is None:
                continue  # defensive; should not happen if any vec exists
            for i in range(emb_dim):
                row[f"iso_e{i}"] = float(iso_vec[i]) 
                row[f"can_e{i}"] = float(can_vec[i])

            rows.append(row)


    out_df = pd.DataFrame(rows)

        
    os.makedirs(save_path, exist_ok=True)
    print(f"Saving to {save_path}/{ds}_{model_name.split('/')[1]}_embeddings.csv")
    out_df.to_csv(f"{save_path}/{ds}_{model_name.split('/')[1]}_embeddings.csv", index=False)



NameError: name 'pd' is not defined

In [5]:



def extract_fingerprints(
    ds: str,
    fp_type: str = "morgan",      # "morgan", "rdkit", or "maccs"
    radius: int = 2,              # for Morgan
    n_bits: int = 1024,           # bit length for Morgan/RDKit
    save_path: str | None = None,
) -> pd.DataFrame:
    """
    Create **SMILES-based fingerprints** from UNIQUE CIDs.

    Assumes the dataset CSV (base_dir/datasets/{ds}/{ds}_data.csv) has:
        - 'cid'
        - 'isomericsmiles'
        - 'canonicalsmiles'

    Output columns:
        ['cid', 'isomeric_text', 'canonical_text',
         'input_type', 'model', 'layer',
         'iso_e0', ..., 'iso_e{d-1}',
         'can_e0', ..., 'can_e{d-1}']

    - 'layer' is always 0 so this stays compatible with your embedding schema.
    - e* columns are now binary fingerprint bits (0/1 or NaN if missing).
    """
    fp_type = fp_type.lower()

    
    if fp_type == "morgan":
        model_name = f"Morgan_r{radius}_{n_bits}"
    elif fp_type == "rdkit":
        model_name = f"RDK_{n_bits}"
    elif fp_type == "maccs":
        model_name = "MACCS"
    else:
        raise ValueError("fp_type must be one of 'morgan', 'rdkit', 'maccs'.")
    print(f"Extracting {fp_type} fingerprints for dataset '{ds}' (SMILES only)...")

    # ---- Load & pick columns ----
    df = pd.read_csv(f"{base_dir}/datasets/{ds}/{ds}_data.csv")

    if "cid" not in df.columns:
        raise ValueError("Dataset must contain a 'cid' column.")
    for col in ("isomericsmiles", "canonicalsmiles"):
        if col not in df.columns:
            raise ValueError(f"Dataset must contain column '{col}' for SMILES fingerprints.")

    iso_col = "isomericsmiles"
    can_col = "canonicalsmiles"

    work = df[["cid", iso_col, can_col]].copy()
    work = (
        work.drop_duplicates(subset=["cid"], keep="first")
            .sort_values("cid")
            .reset_index(drop=True)
    )

    rows = []
    fp_dim = None  # number of bits (set on first successful FP)

    # ---- helper: build fingerprint from SMILES ----
    def make_fp(smiles: str):
        nonlocal fp_dim

        if not isinstance(smiles, str) or not smiles.strip():
            return None
        smiles = smiles.strip()

        mol = Chem.MolFromSmiles(smiles)
        if mol is None:
            return None

        if fp_type == "morgan":
            fp = AllChem.GetMorganFingerprintAsBitVect(mol, radius, nBits=n_bits)
        elif fp_type == "rdkit":
            fp = RDKFingerprint(mol, fpSize=n_bits)
        elif fp_type == "maccs":
            fp = MACCSkeys.GenMACCSKeys(mol)
        else:
            raise ValueError("fp_type must be one of 'morgan', 'rdkit', 'maccs'.")

        num_bits = fp.GetNumBits()
        if fp_dim is None:
            fp_dim = num_bits
        else:
            assert fp_dim == num_bits, "Inconsistent fingerprint length across molecules."

        arr = np.zeros((num_bits,), dtype=int)
        DataStructs.ConvertToNumpyArray(fp, arr)
        return arr

    # ---- Main loop over CIDs ----
    for cid, iso_txt, can_txt in zip(work["cid"], work[iso_col], work[can_col]):
        print(f"Processing CID {cid}...")

        iso_fp = make_fp(iso_txt)
        can_fp = make_fp(can_txt)

        # skip if nothing could be computed
        if iso_fp is None and can_fp is None:
            continue

        # ensure dims match if both exist
        if iso_fp is not None and can_fp is not None:
            assert iso_fp.shape[0] == can_fp.shape[0], "Iso/Can fingerprint dims differ!"

        if fp_dim is None:
            # defensive; shouldn't happen if any fp exists
            continue

        layer_idx = 0  # keep layer axis to match your embedding tables

        row = {
            "cid": cid,
            "isomeric_text": iso_txt if isinstance(iso_txt, str) else "",
            "canonical_text": can_txt if isinstance(can_txt, str) else "",
            "input_type": "smiles",
            "model": model_name,
            "layer": layer_idx,
        }

        # Fill iso_e* and can_e* (bits or NaN)
        for i in range(fp_dim):
            if iso_fp is not None:
                row[f"iso_e{i}"] = int(iso_fp[i])
            else:
                row[f"iso_e{i}"] = np.nan

            if can_fp is not None:
                row[f"can_e{i}"] = int(can_fp[i])
            else:
                row[f"can_e{i}"] = np.nan

        rows.append(row)

    out_df = pd.DataFrame(rows)

    if save_path is not None:
        os.makedirs(save_path, exist_ok=True)
        fname = f"{save_path}/{ds}_{model_name}_embeddings.csv"
        print(f"Saving to {fname}")
        out_df.to_csv(fname, index=False)

    return out_df


In [20]:
def extract_rdkit_descriptors(
    ds: str,
    save_path: str | None = None,
    model_name: str = "RDKit_Descriptors",
) -> pd.DataFrame:
    """
    Extract **all available RDKit molecular descriptors** for SMILES-only datasets.

    Output columns:
        ['cid', 'isomeric_text', 'canonical_text', 'input_type', 'model', 'layer',
         'iso_DESCNAME1', ..., 'iso_DESCNAMEN',
         'can_DESCNAME1', ..., 'can_DESCNAMEN']
    """

    print(f"Extracting full RDKit descriptor set for dataset '{ds}' (SMILES only)...")

    # ---- Load dataset ----
    df = pd.read_csv(f"{base_dir}/datasets/{ds}/{ds}_data.csv")

    if "cid" not in df.columns:
        raise ValueError("Dataset must contain a 'cid' column.")

    for col in ("isomericsmiles", "canonicalsmiles"):
        if col not in df.columns:
            raise ValueError(f"Dataset must contain column '{col}' for SMILES descriptors.")

    iso_col = "isomericsmiles"
    can_col = "canonicalsmiles"

    work = df[["cid", iso_col, can_col]].drop_duplicates("cid").sort_values("cid").reset_index(drop=True)

    # ---- Descriptor names ----
    descriptor_names = [desc_name for desc_name, _ in Descriptors._descList]

    calc = MoleculeDescriptors.MolecularDescriptorCalculator(descriptor_names)

    rows = []

    # ---- Helper ----
    def compute_descriptors(smiles):
        """Return list of descriptor values or None if molecule cannot be parsed."""
        if not isinstance(smiles, str) or not smiles.strip():
            return None
        mol = Chem.MolFromSmiles(smiles.strip())
        if mol is None:
            return None
        try:
            vals = calc.CalcDescriptors(mol)
            return list(vals)
        except Exception:
            return None

    # ---- Main loop ----
    for cid, iso_txt, can_txt in zip(work["cid"], work[iso_col], work[can_col]):
        print(f"Processing CID {cid}...")

        iso_desc = compute_descriptors(iso_txt)
        can_desc = compute_descriptors(can_txt)

        # skip if no descriptors could be computed
        if iso_desc is None and can_desc is None:
            continue

        row = {
            "cid": cid,
            "isomeric_text": iso_txt if isinstance(iso_txt, str) else "",
            "canonical_text": can_txt if isinstance(can_txt, str) else "",
            "input_type": "smiles",
            "model": model_name,
            "layer": 0,  # to match your existing schema
        }

        # Fill descriptor columns
        for name_idx, desc_name in enumerate(descriptor_names):
            iso_val = iso_desc[name_idx] if iso_desc is not None else np.nan
            can_val = can_desc[name_idx] if can_desc is not None else np.nan

            row[f"iso_{desc_name}"] = iso_val
            row[f"can_{desc_name}"] = can_val

        rows.append(row)

    out_df = pd.DataFrame(rows)

    # ---- Save ----
    if save_path is not None:
        os.makedirs(save_path, exist_ok=True)
        fname = f"{save_path}/{ds}_{model_name}.csv"
        print(f"Saving RDKit descriptor table to: {fname}")
        out_df.to_csv(fname, index=False)

    return out_df

In [ ]:


def extract_attention_weights(
    tokenizer,
    model,
    model_name: str,
    ds: str,
    input_type: str = "smiles",  # "smiles" or "selfies"
    token: int = 0,              # which query token index to use (e.g. CLS at 0)
    save_path: str | None = None,
) -> pd.DataFrame:
    """
    Extract attention weights from UNIQUE CIDs using either SMILES or SELFIES.

    For each CID and each layer, we store:
        - head-averaged attention distribution from `token` -> all tokens
        - separately for isomeric and canonical text (if available)

    Output DF columns:
        ['cid', 'isomeric_text', 'canonical_text', 'input_type', 'model', 'layer',
         'iso_att', 'can_att']

    where 'iso_att' and 'can_att' are Python lists (variable length, depends on seq len).
    """
    assert input_type.lower() in {"smiles", "selfies"}, "input_type must be 'smiles' or 'selfies'"
    model.eval()
    print(f"Extracting {model_name} ATTENTIONS for {ds} using {input_type}...")

    # ---- Load & pick columns ----
    df = pd.read_csv(f"{base_dir}/datasets/{ds}/{ds}_data.csv")
    if "cid" not in df.columns:
        raise ValueError("Dataset must contain a 'cid' column.")

    if input_type.lower() == "smiles":
        iso_col = "isomericsmiles"
        can_col = "canonicalsmiles"
    else:
        iso_col = "isomericselfies"
        can_col = "canonicalselfies"

    use_cols = ["cid"]
    if iso_col in df.columns:
        use_cols.append(iso_col)
    if can_col in df.columns:
        use_cols.append(can_col)

    work = (
        df[use_cols]
        .drop_duplicates(subset=["cid"], keep="first")
        .sort_values("cid")
        .reset_index(drop=True)
    )

    # ---- Device ----
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    rows = []

    for _, row_df in work.iterrows():
        cid = row_df["cid"]
        iso_txt = row_df.get(iso_col, "")
        can_txt = row_df.get(can_col, "")

        print(f"Processing CID {cid}...")

        # normalize texts
        iso_txt = iso_txt if isinstance(iso_txt, str) and iso_txt.strip() else ""
        can_txt = can_txt if isinstance(can_txt, str) and can_txt.strip() else ""
        if not iso_txt and not can_txt:
            continue

        # --- run model for isomeric (if present) ---
        iso_atts = None
        if iso_txt:
            iso_inputs = tokenizer([iso_txt], padding=True, truncation=True, return_tensors="pt")
            iso_inputs = {k: v.to(device) for k, v in iso_inputs.items()}
            with torch.no_grad():
                iso_out = model(**iso_inputs, output_attentions=True)
            iso_atts = iso_out.attentions  # tuple of length n_layers, each (B, H, T, T)

        # --- run model for canonical (if present) ---
        can_atts = None
        if can_txt:
            can_inputs = tokenizer([can_txt], padding=True, truncation=True, return_tensors="pt")
            can_inputs = {k: v.to(device) for k, v in can_inputs.items()}
            with torch.no_grad():
                can_out = model(**can_inputs, output_attentions=True)
            can_atts = can_out.attentions

        n_layers = max(
            len(iso_atts) if iso_atts is not None else 0,
            len(can_atts) if can_atts is not None else 0,
        )

        for layer_idx in range(n_layers):
            iso_att_vec = None
            can_att_vec = None

            # ---- isomeric attention: CLS/token -> all tokens, averaged over heads ----
            if iso_atts is not None:
                att = iso_atts[layer_idx]  # (1, H, T, T)
                # make sure token index is valid
                T = att.shape[-1]
                q_idx = token if token < T else 0
                # average over heads -> (T, T), then pick row for q_idx -> (T,)
                att_avg = att[0].mean(dim=0)           # (T, T)
                iso_att_vec = att_avg[q_idx].detach().cpu().numpy()  # (T,)

            # ---- canonical attention: CLS/token -> all tokens, averaged over heads ----
            if can_atts is not None:
                att = can_atts[layer_idx]  # (1, H, T, T)
                T = att.shape[-1]
                q_idx = token if token < T else 0
                att_avg = att[0].mean(dim=0)           # (T, T)
                can_att_vec = att_avg[q_idx].detach().cpu().numpy()  # (T,)

            row = {
                "cid": cid,
                "isomeric_text": iso_txt,
                "canonical_text": can_txt,
                "input_type": input_type.lower(),
                "model": model_name,
                "layer": layer_idx,
                # store as lists; seq len can differ per CID
                "iso_att": iso_att_vec.tolist() if iso_att_vec is not None else None,
                "can_att": can_att_vec.tolist() if can_att_vec is not None else None,
            }
            rows.append(row)

    out_df = pd.DataFrame(rows)

    if save_path is not None:
        os.makedirs(save_path, exist_ok=True)
        out_file = f"{save_path}/{ds}_{model_name.split('/')[-1]}_attentions.csv"
        print(f"Saving to {out_file}")
        out_df.to_csv(out_file, index=False)

    return out_df


In [ ]:


@torch.inference_mode()
def extract_representations_pairs(
    tokenizer,
    model,
    model_name: str,
    ds: str,
    input_type: str = "smiles",   # "smiles" or "selfies"
    token: int = 0,               # CLS/first token index
    save_path: Optional[str] = None,
) -> pd.DataFrame:
    """
    Extract per-layer embeddings for BOTH stimuli in a pairwise similarity dataset.

    Expected columns:
        cid stimulus 1, cid stimulus 2, similarity,
        isomericsmiles stimulus 1, isomericsmiles stimulus 2,
        canonicalsmiles stimulus 1, canonicalsmiles stimulus 2,
        isomericselfies stimulus 1, isomericselfies stimulus 2,
        canonicalselfies stimulus 1, canonicalselfies stimulus 2

    Output columns:
        ['pair_index','stimulus_id','cid','stimulus','isomeric_text','canonical_text',
         'input_type','model','layer','e0','e1',...,'e{d-1}']
    """
    assert input_type.lower() in {"smiles", "selfies"}, "input_type must be 'smiles' or 'selfies'"
    model.eval()
    print(f"Extracting {model_name} embeddings for {ds} (pairwise mode, {input_type})...")

    # ---- Load ----
    df = pd.read_csv(f"../datasets/{ds}/{ds}_data.csv")
    required_cols = [f"cid stimulus 1", f"cid stimulus 2"]
    if not all(c in df.columns for c in required_cols):
        raise ValueError(f"File must include: {required_cols}")

    # ---- Column mapping ----
    if input_type.lower() == "smiles":
        iso1_col, iso2_col = "isomericsmiles stimulus 1", "isomericsmiles stimulus 2"
        can1_col, can2_col = "canonicalsmiles stimulus 1", "canonicalsmiles stimulus 2"
    else:
        iso1_col, iso2_col = "isomericselfies stimulus 1", "isomericselfies stimulus 2"
        can1_col, can2_col = "canonicalselfies stimulus 1", "canonicalselfies stimulus 2"

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    rows = []
    emb_dim = None

    # iterate through each pair
    for i, row in df.iterrows():
        for stim_idx in [1, 2]:
            cid = row[f"cid stimulus {stim_idx}"]
            iso_txt = row.get(f"{'isomeric' + input_type} stimulus {stim_idx}", None)
            can_txt = row.get(f"canonical{input_type} stimulus {stim_idx}", None)

            iso_txt = str(iso_txt).strip() if isinstance(iso_txt, str) else ""
            can_txt = str(can_txt).strip() if isinstance(can_txt, str) else ""
            if not iso_txt and not can_txt:
                continue

            # tokenize and run model for both iso and can if available
            def run_encoder(txt: str):
                if not txt: 
                    return None
                inputs = tokenizer([txt], padding=True, truncation=True, return_tensors="pt").to(device)
                out = model(**inputs, output_hidden_states=True)
                return out.hidden_states  # list of [B, T, D]

            iso_hiddens = run_encoder(iso_txt)
            can_hiddens = run_encoder(can_txt)

            n_layers = max(
                len(iso_hiddens) if iso_hiddens is not None else 0,
                len(can_hiddens) if can_hiddens is not None else 0,
            )

            for layer_idx in range(n_layers):
                iso_vec = iso_hiddens[layer_idx][0, token, :].detach().cpu().numpy() if iso_hiddens else None
                can_vec = can_hiddens[layer_idx][0, token, :].detach().cpu().numpy() if can_hiddens else None

                if emb_dim is None:
                    emb_dim = iso_vec.shape[0] if iso_vec is not None else can_vec.shape[0]

                row_out = {
                    "pair_index": i,
                    "stimulus": stim_idx,
                    "cid": cid,
                    "isomeric_text": iso_txt,
                    "canonical_text": can_txt,
                    "input_type": input_type.lower(),
                    "model": model_name,
                    "layer": layer_idx,
                }

                # fill embedding vector
                if iso_vec is not None:
                    for j in range(emb_dim):
                        row_out[f"iso_e{j}"] = float(iso_vec[j])
                if can_vec is not None:
                    for j in range(emb_dim):
                        row_out[f"can_e{j}"] = float(can_vec[j])

                rows.append(row_out)

        if (i + 1) % 20 == 0:
            print(f"Processed {i+1}/{len(df)} pairs...")

    out_df = pd.DataFrame(rows)
    os.makedirs(save_path, exist_ok=True)
    out_file = f"{save_path}/{ds}_{model_name.split('/')[-1]}_pair_embeddings.csv"
    out_df.to_csv(out_file, index=False)
    print(f"Saved embeddings → {out_file}")
    return out_df


In [25]:

def extract_representations_per_token(
    tokenizer,
    model,
    model_name: str,
    ds: str,
    input_type: str = "smiles",   # "smiles" or "selfies"
    save_path: str | None = None,
) -> pd.DataFrame:
    """
    For each CID, layer, and token:
        -> one row with the embedding of that token at that layer.

    Columns:
        - cid
        - isomeric_text
        - canonical_text
        - used_text          (the string that was actually tokenized)
        - used_variant       ("iso" or "can")
        - input_type         ("smiles" / "selfies")
        - model              (model_name)
        - layer              (int, layer index as returned by model)
        - token_idx          (position in the sequence)
        - token_id           (vocab ID)
        - token_str          (token text from tokenizer)
        - e0 ... e{d-1}      (embedding dimensions)
    """

    assert input_type.lower() in {"smiles", "selfies"}, "input_type must be 'smiles' or 'selfies'"
    print(f"Extracting {model_name} PER-TOKEN embeddings for {ds} using {input_type}...")

    # ---- Load & pick columns ----
    df = pd.read_csv(f"{base_dir}/datasets/{ds}/{ds}_data.csv")
    if "cid" not in df.columns:
        raise ValueError("Dataset must contain a 'cid' column.")

    if input_type.lower() == "smiles":
        iso_col = "isomericsmiles"
        can_col = "canonicalsmiles"
    else:  # selfies
        iso_col = "isomericselfies"
        can_col = "canonicalselfies"

    if iso_col is None and can_col is None:
        raise ValueError(f"No {input_type} columns found.")

    use_cols = ["cid"]
    if iso_col: use_cols.append(iso_col)
    if can_col: use_cols.append(can_col)

    work = (
        df[use_cols]
        .drop_duplicates(subset=["cid"], keep="first")
        .sort_values("cid")
        .reset_index(drop=True)
    )

    # ---- Device ----
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    model.eval()

    rows = []
    emb_dim = None

    for _, row_in in work.iterrows():
        cid = row_in["cid"]
        iso_txt = row_in.get(iso_col, "")
        can_txt = row_in.get(can_col, "")

        print(f"Processing CID {cid}...")

        # normalize texts
        iso_txt = iso_txt if isinstance(iso_txt, str) and iso_txt.strip() else ""
        can_txt = can_txt if isinstance(can_txt, str) and can_txt.strip() else ""

        # choose which text to encode: prefer isomeric, fallback to canonical
        if iso_txt:
            used_text = iso_txt
            used_variant = "iso"
        elif can_txt:
            used_text = can_txt
            used_variant = "can"
        else:
            # nothing to encode
            continue

        # ---- Tokenize & run model ----
        inputs = tokenizer(
            [used_text],
            padding=True,
            truncation=True,
            return_tensors="pt"
        )
        inputs = {k: v.to(device) for k, v in inputs.items()}

        with torch.no_grad():
            out = model(**inputs, output_hidden_states=True)

        hidden_states = out.hidden_states  # tuple of length n_layers, each (1, T, D)
        input_ids = inputs["input_ids"][0]  # (T,)
        T = input_ids.shape[0]
        n_layers = len(hidden_states)

        # Set emb_dim once
        if emb_dim is None:
            emb_dim = hidden_states[0].shape[-1]

        # ---- Loop over layers and tokens ----
        for layer_idx in range(n_layers):
            h = hidden_states[layer_idx][0]  # (T, D)

            for token_idx in range(T):
                token_id = int(input_ids[token_idx].item())
                token_str = tokenizer.convert_ids_to_tokens([token_id])[0]

                vec = h[token_idx].detach().cpu().numpy()  # (D,)

                row = {
                    "cid": cid,
                    "isomeric_text": iso_txt,
                    "canonical_text": can_txt,
                    "used_text": used_text,
                    "used_variant": used_variant,  # "iso" / "can"
                    "input_type": input_type.lower(),
                    "model": model_name,
                    "layer": layer_idx,
                    "token_idx": token_idx,
                    "token_id": token_id,
                    "token_str": token_str,
                }

                # add embedding dims
                for i in range(emb_dim):
                    row[f"e{i}"] = float(vec[i])

                rows.append(row)

    out_df = pd.DataFrame(rows)

    if save_path is not None:
        os.makedirs(save_path, exist_ok=True)
        out_file = f"{save_path}/{ds}_{model_name.split('/')[1]}_per_token_embeddings.csv"
        print(f"Saving to {out_file}")
        out_df.to_csv(out_file, index=False)

    return out_df


In [3]:
from typing import Optional, Literal, Tuple

def build_cosine_similarity_from_embeddings(ds,model_name,
    *,
    layer: Literal["last", "mean"] | int = "last",
    variant: Literal["iso", "can", "prefer_iso", "prefer_can"] = "prefer_iso",
    out_csv: Optional[str] = None,
    overwrite_similarity: bool = True,
    verbose: bool = True,
) -> pd.DataFrame:
    """
    Compute cosine similarity between stimulus 1 and 2 using precomputed embeddings
    from `extract_representations_pairs` and rebuild the pairwise CSV.

    Parameters
    ----------
    embeddings_csv : str
        CSV produced by extract_representations_pairs (with columns:
        pair_index, stimulus, layer, iso_e*, can_e*, ...).
    original_pairs_csv : str
        Your original pairwise file (cid stimulus 1/2, names, SMILES/SELFIES, ...).
    layer : {"last","mean"} or int, default "last"
        - "last": use max available layer per (pair_index, stimulus).
        - "mean": average vectors across all available layers.
        - int: use that specific layer index.
    variant : {"iso","can","prefer_iso","prefer_can"}, default "prefer_iso"
        Which embedding to use. "prefer_*" falls back to the other if missing.
    out_csv : Optional[str]
        Where to save the result. If None, writes next to `original_pairs_csv`
        with suffix `_cosine_from_embs.csv`.
    overwrite_similarity : bool, default True
        If True, replaces column `similarity`. If False, writes `similarity_cosine`.
    verbose : bool, default True
        Print progress.

    Returns
    -------
    pd.DataFrame
        The rebuilt DataFrame with cosine similarities written to disk.
    """
    # ---- Load files ----
    original_pairs_csv=f"../datasets/{ds}/{ds}_data.csv"
    embeddings_csv=f"embeddings/{ds}_{model_name.split('/')[1]}_pair_embeddings.csv"
    embs = pd.read_csv(embeddings_csv)
    pairs = pd.read_csv(original_pairs_csv)

    # Basic checks
    for col in ["pair_index", "stimulus", "layer"]:
        if col not in embs.columns:
            raise ValueError(f"Embeddings CSV must contain '{col}' (did it come from extract_representations_pairs?).")

    # Identify embedding columns present
    iso_cols = [c for c in embs.columns if c.startswith("iso_e")]
    can_cols = [c for c in embs.columns if c.startswith("can_e")]
    if not iso_cols and not can_cols:
        raise ValueError("Embeddings CSV has no 'iso_e*' nor 'can_e*' columns.")

    # Helper: extract a vector for a given (pair_index, stimulus)
    def _subset_for(pid: int, stim: int) -> pd.DataFrame:
        return embs[(embs["pair_index"] == pid) & (embs["stimulus"] == stim)]

    def _vec_from_rows(rows: pd.DataFrame) -> Tuple[Optional[np.ndarray], Optional[np.ndarray]]:
        if rows.empty:
            return None, None

        # choose rows by layer
        if layer == "last":
            lyr = rows["layer"].max()
            rows = rows[rows["layer"] == lyr]
        elif layer == "mean":
            pass  # keep all layers
        else:
            lyr = int(layer)
            rows = rows[rows["layer"] == lyr]
            if rows.empty:
                return None, None

        v_iso = None
        v_can = None

        if iso_cols:
            arr_iso = rows[iso_cols].to_numpy(dtype=float)  # (L_sel, D) or (1, D)
            if not np.isnan(arr_iso).all():
                if layer == "mean":
                    v_iso = np.nanmean(arr_iso, axis=0)
                else:
                    v_iso = arr_iso[0]
                if np.isnan(v_iso).all():
                    v_iso = None

        if can_cols:
            arr_can = rows[can_cols].to_numpy(dtype=float)
            if not np.isnan(arr_can).all():
                if layer == "mean":
                    v_can = np.nanmean(arr_can, axis=0)
                else:
                    v_can = arr_can[0]
                if np.isnan(v_can).all():
                    v_can = None

        return v_iso, v_can

    def _choose_variant(v_iso: Optional[np.ndarray], v_can: Optional[np.ndarray]) -> Optional[np.ndarray]:
        if variant == "iso":
            return v_iso
        if variant == "can":
            return v_can
        if variant == "prefer_iso":
            return v_iso if v_iso is not None else v_can
        if variant == "prefer_can":
            return v_can if v_can is not None else v_iso
        return None

    def _cosine(a: Optional[np.ndarray], b: Optional[np.ndarray]) -> float:
        if a is None or b is None:
            return float("nan")
        na = np.linalg.norm(a)
        nb = np.linalg.norm(b)
        if na == 0.0 or nb == 0.0 or not np.isfinite(na) or not np.isfinite(nb):
            return float("nan")
        return float(np.dot(a, b) / (na * nb))

    # ---- Compute per-pair cosine ----
    sims = []
    n = len(pairs)
    for i, _ in pairs.iterrows():
        rows1 = _subset_for(i, 1)
        rows2 = _subset_for(i, 2)
        v1_iso, v1_can = _vec_from_rows(rows1)
        v2_iso, v2_can = _vec_from_rows(rows2)

        v1 = _choose_variant(v1_iso, v1_can)
        v2 = _choose_variant(v2_iso, v2_can)
        sim = _cosine(v1, v2)
        sims.append(sim)

        if verbose and (i + 1) % 50 == 0:
            print(f"Processed {i+1}/{n} pairs...")

    out = pairs.copy()
    target_col = "similarity" if overwrite_similarity else "similarity_cosine"
    out[target_col] = sims

    # ---- Save ----
    if out_csv is None:
        base, ext = os.path.splitext(original_pairs_csv)
        out_csv = f"{base}_{model_name.split('/')[1]}_cosine_from_embs_{variant}.csv"
    out.to_csv(out_csv, index=False)
    if verbose:
        print(f"Saved cosine-similarity file → {out_csv}")

    return out

In [ ]:
# def extract_representations_by_molfeat(model_name,transformer,input_type='smiles',token=0):
    
#     for subject_id in range(s_start,s_end+1):
#         input_molecules = pd.read_csv(f'{base_dir}/fmri/embeddings{ds}/CIDs_smiles_selfies_{subject_id}{ds}.csv')[input_type].values.tolist()
        
#         outputs = transformer(input_molecules)
#         np.save(f'{base_dir}/fmri/embeddings{ds}/embeddings_{model_name}_{subject_id}_{-1}{ds}.npy',outputs)

# Encoder-Only

### Token/

In [24]:
Input_types = {
    'ibm/MoLFormer-XL-both-10pct':'smiles',
    'seyonec/ChemBERTa-zinc-base-v1':'smiles',
    'jonghyunlee/ChemBERT_ChEMBL_pretrained':'smiles',
    "HUBioDataLab/SELFormer":'selfies'
    }

for model_name in ['ibm/MoLFormer-XL-both-10pct',
                   'seyonec/ChemBERTa-zinc-base-v1',"jonghyunlee/ChemBERT_ChEMBL_pretrained","HUBioDataLab/SELFormer"
                   ]:
    for ds in [
        'sagar2023'
        # ,'keller2016',
        # 'bierling2025',
            #    'leffingwell'
               ]:
    
        model = AutoModel.from_pretrained(model_name, trust_remote_code=True,use_safetensors=True)
        tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
        # extract_representations(tokenizer, model,model_name,save_path='avgtokenembeddings',ds=ds,input_type=Input_types[model_name],mean=True)
        vocab = tokenizer.get_vocab()

        # convert to id → token
        id_to_token = {id_: tok for tok, id_ in vocab.items()}

        # make DataFrame
        df = pd.DataFrame({
            "token_id": list(id_to_token.keys()),
            "token": list(id_to_token.values())
        })

        # sort by token_id
        df = df.sort_values("token_id").reset_index(drop=True)

        # save CSV
        fname = f"tokens/{model_name.split('/')[-1]}_vocab.csv"
        df.to_csv(fname, index=False)
        # extract_tokens(tokenizer, model,model_name,save_path='tokens',ds=ds,input_type=Input_types[model_name])



Some weights of RobertaModel were not initialized from the model checkpoint at seyonec/ChemBERTa-zinc-base-v1 and are newly initialized: ['embeddings.word_embeddings.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of BertModel were not initialized from the model checkpoint at jonghyunlee/ChemBERT_ChEMBL_pretrained and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at HUBioDataLab/SELFormer and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


## MoLFormer-XL-both-10pct

In [28]:
Input_types = {
    'ibm/MoLFormer-XL-both-10pct':'smiles',
    'seyonec/ChemBERTa-zinc-base-v1':'smiles',
    'jonghyunlee/ChemBERT_ChEMBL_pretrained':'smiles',
    "HUBioDataLab/SELFormer":'selfies'
    }

for model_name in ['ibm/MoLFormer-XL-both-10pct',
                   'seyonec/ChemBERTa-zinc-base-v1',"jonghyunlee/ChemBERT_ChEMBL_pretrained","HUBioDataLab/SELFormer"
                   ]:
    for ds in [
        'sagar2023'
        # ,'keller2016',
        # 'bierling2025',
            #    'leffingwell'
               ]:
    
        model = AutoModel.from_pretrained(model_name, trust_remote_code=True,use_safetensors=True)
        tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
        # extract_representations(tokenizer, model,model_name,save_path='avgtokenembeddings',ds=ds,input_type=Input_types[model_name],mean=True)
        extract_representations_per_token(tokenizer, model,model_name,save_path='embeddingstoken',ds=ds,input_type=Input_types[model_name])



Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Extracting ibm/MoLFormer-XL-both-10pct PER-TOKEN embeddings for sagar2023 using smiles...
Processing CID 126...
Processing CID 177...
Processing CID 196...
Processing CID 239...
Processing CID 240...
Processing CID 261...
Processing CID 263...
Processing CID 325...
Processing CID 326...
Processing CID 356...
Processing CID 379...
Processing CID 460...
Processing CID 650...
Processing CID 660...
Processing CID 957...
Processing CID 1001...
Processing CID 1032...
Processing CID 1049...
Processing CID 1068...
Processing CID 1110...
Processing CID 1136...
Processing CID 2214...
Processing CID 2345...
Processing CID 2346...
Processing CID 2758...
Processing CID 2879...
Processing CID 2969...
Processing CID 3893...
Processing CID 4133...
Processing CID 5541...
Processing CID 5610...
Processing CID 5779...
Processing CID 5780...
Processing CID 5960...
Processing CID 6050...
Processing CID 6054...
Processing CID 6106...
Processing CID 6184...
Processing CID 6448...
Processing CID 6501...
Proce

Some weights of RobertaModel were not initialized from the model checkpoint at seyonec/ChemBERTa-zinc-base-v1 and are newly initialized: ['embeddings.word_embeddings.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Extracting seyonec/ChemBERTa-zinc-base-v1 PER-TOKEN embeddings for sagar2023 using smiles...
Processing CID 126...
Processing CID 177...
Processing CID 196...
Processing CID 239...
Processing CID 240...
Processing CID 261...
Processing CID 263...
Processing CID 325...
Processing CID 326...
Processing CID 356...
Processing CID 379...
Processing CID 460...
Processing CID 650...
Processing CID 660...
Processing CID 957...
Processing CID 1001...
Processing CID 1032...
Processing CID 1049...
Processing CID 1068...
Processing CID 1110...
Processing CID 1136...
Processing CID 2214...
Processing CID 2345...
Processing CID 2346...
Processing CID 2758...
Processing CID 2879...
Processing CID 2969...
Processing CID 3893...
Processing CID 4133...
Processing CID 5541...
Processing CID 5610...
Processing CID 5779...
Processing CID 5780...
Processing CID 5960...
Processing CID 6050...
Processing CID 6054...
Processing CID 6106...
Processing CID 6184...
Processing CID 6448...
Processing CID 6501...
Pr

Some weights of BertModel were not initialized from the model checkpoint at jonghyunlee/ChemBERT_ChEMBL_pretrained and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Extracting jonghyunlee/ChemBERT_ChEMBL_pretrained PER-TOKEN embeddings for sagar2023 using smiles...
Processing CID 126...
Processing CID 177...
Processing CID 196...
Processing CID 239...
Processing CID 240...
Processing CID 261...
Processing CID 263...
Processing CID 325...
Processing CID 326...
Processing CID 356...
Processing CID 379...
Processing CID 460...
Processing CID 650...
Processing CID 660...
Processing CID 957...
Processing CID 1001...
Processing CID 1032...
Processing CID 1049...
Processing CID 1068...
Processing CID 1110...
Processing CID 1136...
Processing CID 2214...
Processing CID 2345...
Processing CID 2346...
Processing CID 2758...
Processing CID 2879...
Processing CID 2969...
Processing CID 3893...
Processing CID 4133...
Processing CID 5541...
Processing CID 5610...
Processing CID 5779...
Processing CID 5780...
Processing CID 5960...
Processing CID 6050...
Processing CID 6054...
Processing CID 6106...
Processing CID 6184...
Processing CID 6448...
Processing CID 65

Some weights of RobertaModel were not initialized from the model checkpoint at HUBioDataLab/SELFormer and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Extracting HUBioDataLab/SELFormer PER-TOKEN embeddings for sagar2023 using selfies...
Processing CID 126...
Processing CID 177...
Processing CID 196...
Processing CID 239...
Processing CID 240...
Processing CID 261...
Processing CID 263...
Processing CID 325...
Processing CID 326...
Processing CID 356...
Processing CID 379...
Processing CID 460...
Processing CID 650...
Processing CID 660...
Processing CID 957...
Processing CID 1001...
Processing CID 1032...
Processing CID 1049...
Processing CID 1068...
Processing CID 1110...
Processing CID 1136...
Processing CID 2214...
Processing CID 2345...
Processing CID 2346...
Processing CID 2758...
Processing CID 2879...
Processing CID 2969...
Processing CID 3893...
Processing CID 4133...
Processing CID 5541...
Processing CID 5610...
Processing CID 5779...
Processing CID 5780...
Processing CID 5960...
Processing CID 6050...
Processing CID 6054...
Processing CID 6106...
Processing CID 6184...
Processing CID 6448...
Processing CID 6501...
Processin

In [17]:
for fp_type in ['morgan','rdkit','maccs']:
    for ds in [
        'sagar2023'
        # ,'keller2016',
        # 'bierling2025',
            #    'leffingwell'
               ]:
        extract_fingerprints(
            ds=ds,
            fp_type=fp_type,
            radius=2,
            n_bits=1024,
            save_path="fingerprints",
        )

Extracting morgan fingerprints for dataset 'sagar2023' (SMILES only)...
Processing CID 126...
Processing CID 177...
Processing CID 196...
Processing CID 239...
Processing CID 240...
Processing CID 261...
Processing CID 263...
Processing CID 325...
Processing CID 326...
Processing CID 356...
Processing CID 379...
Processing CID 460...
Processing CID 650...
Processing CID 660...
Processing CID 957...
Processing CID 1001...
Processing CID 1032...
Processing CID 1049...
Processing CID 1068...
Processing CID 1110...
Processing CID 1136...
Processing CID 2214...
Processing CID 2345...
Processing CID 2346...
Processing CID 2758...
Processing CID 2879...
Processing CID 2969...
Processing CID 3893...
Processing CID 4133...
Processing CID 5541...
Processing CID 5610...
Processing CID 5779...
Processing CID 5780...
Processing CID 5960...
Processing CID 6050...
Processing CID 6054...
Processing CID 6106...
Processing CID 6184...
Processing CID 6448...
Processing CID 6501...
Processing CID 6549...


In [21]:
for ds in [
        'sagar2023'
        # ,'keller2016',
        # 'bierling2025',
            #    'leffingwell'
               ]:
    extract_rdkit_descriptors(
        ds=ds,
        save_path="rdkit_descriptors",
        model_name="RDKit_Descriptors",
    )

Extracting full RDKit descriptor set for dataset 'sagar2023' (SMILES only)...
Processing CID 126...
Processing CID 177...
Processing CID 196...
Processing CID 239...
Processing CID 240...
Processing CID 261...
Processing CID 263...
Processing CID 325...
Processing CID 326...
Processing CID 356...
Processing CID 379...
Processing CID 460...
Processing CID 650...
Processing CID 660...
Processing CID 957...
Processing CID 1001...
Processing CID 1032...
Processing CID 1049...
Processing CID 1068...
Processing CID 1110...
Processing CID 1136...
Processing CID 2214...
Processing CID 2345...
Processing CID 2346...
Processing CID 2758...
Processing CID 2879...
Processing CID 2969...
Processing CID 3893...
Processing CID 4133...
Processing CID 5541...
Processing CID 5610...
Processing CID 5779...
Processing CID 5780...
Processing CID 5960...
Processing CID 6050...
Processing CID 6054...
Processing CID 6106...
Processing CID 6184...
Processing CID 6448...
Processing CID 6501...
Processing CID 65

In [7]:
Input_types = {
    'ibm/MoLFormer-XL-both-10pct':'smiles',
    'seyonec/ChemBERTa-zinc-base-v1':'smiles',
    'jonghyunlee/ChemBERT_ChEMBL_pretrained':'smiles',
    "HUBioDataLab/SELFormer":'selfies'
    }

for model_name in ['ibm/MoLFormer-XL-both-10pct',
    # 'seyonec/ChemBERTa-zinc-base-v1',"jonghyunlee/ChemBERT_ChEMBL_pretrained","HUBioDataLab/SELFormer"
    ]:
    for ds in [
        'sagar2023',
        # 'keller2016','bierling2025',
            #    'leffingwell'
               ]:
    
        model = AutoModel.from_pretrained(model_name, trust_remote_code=True,use_safetensors=True)
        tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
        extract_attention_weights(tokenizer, model,model_name,save_path='attention',ds=ds,input_type=Input_types[model_name])



NameError: name 'extract_attention_weights' is not defined

In [ ]:
Input_types = {
    'ibm/MoLFormer-XL-both-10pct':'smiles',
    'seyonec/ChemBERTa-zinc-base-v1':'smiles',
    'jonghyunlee/ChemBERT_ChEMBL_pretrained':'smiles',
    "HUBioDataLab/SELFormer":'selfies'
    }

for model_name in ['ibm/MoLFormer-XL-both-10pct','seyonec/ChemBERTa-zinc-base-v1',"jonghyunlee/ChemBERT_ChEMBL_pretrained","HUBioDataLab/SELFormer"]:
    for ds in [
        # 'sagar2023','keller2016','bierling2025',
               'snitz2013']:
    
        model = AutoModel.from_pretrained(model_name, trust_remote_code=True,use_safetensors=True)
        tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
        extract_representations_pairs(tokenizer, model,model_name,save_path='embeddings',ds=ds,input_type=Input_types[model_name])



Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Extracting ibm/MoLFormer-XL-both-10pct embeddings for snitz2013 (pairwise mode, smiles)...


KeyboardInterrupt: 

In [21]:
for model_name in ['ibm/MoLFormer-XL-both-10pct','seyonec/ChemBERTa-zinc-base-v1',"jonghyunlee/ChemBERT_ChEMBL_pretrained","HUBioDataLab/SELFormer"]:
    for ds in [
        # 'sagar2023','keller2016','bierling2025',
               'snitz2013']:
        out = build_cosine_similarity_from_embeddings(ds,model_name,
            # embeddings_csv=f"embeddings/{ds}_{model_name.split('/')[1]}_pair_embeddings.csv",
            layer="last",
            variant="prefer_can",
            out_csv=None,
            overwrite_similarity=True,
            verbose=True)

Processed 50/83 pairs...
Saved cosine-similarity file → ../datasets/snitz2013/snitz2013_data_MoLFormer-XL-both-10pct_cosine_from_embs_prefer_can.csv
Processed 50/83 pairs...
Saved cosine-similarity file → ../datasets/snitz2013/snitz2013_data_ChemBERTa-zinc-base-v1_cosine_from_embs_prefer_can.csv
Processed 50/83 pairs...
Saved cosine-similarity file → ../datasets/snitz2013/snitz2013_data_ChemBERT_ChEMBL_pretrained_cosine_from_embs_prefer_can.csv
Processed 50/83 pairs...
Saved cosine-similarity file → ../datasets/snitz2013/snitz2013_data_SELFormer_cosine_from_embs_prefer_can.csv
